## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_33.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_33.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [ ]:
# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 4000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.2),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 100.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 100.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-17 00:42:29,142] A new study created in memory with name: no-name-3cc72d97-f1b6-4500-8036-f22f010104c8
[I 2025-02-17 00:42:53,203] Trial 0 finished with value: 0.7178154550635842 and parameters: {'n_estimators': 1931, 'num_leaves': 2035, 'learning_rate': 0.06798491346395485, 'min_child_samples': 52, 'reg_alpha': 6.049592394722953, 'reg_lambda': 7.914086467979773}. Best is trial 0 with value: 0.7178154550635842.
[I 2025-02-17 00:44:41,723] Trial 1 finished with value: 0.6932855918955614 and parameters: {'n_estimators': 2630, 'num_leaves': 3327, 'learning_rate': 0.07853011761736381, 'min_child_samples': 309, 'reg_alpha': 1.3740355845706276, 'reg_lambda': 7.940918429667102}. Best is trial 0 with value: 0.7178154550635842.
[I 2025-02-17 00:44:54,605] Trial 2 finished with value: 0.7189489578314743 and parameters: {'n_estimators': 1952, 'num_leaves': 1039, 'learning_rate': 0.17463321079921748, 'min_child_samples': 221, 'reg_alpha': 7.374133411258543, 'reg_lambda': 7.6744571902862

KeyboardInterrupt: 

[I 2025-02-16 14:01:21,962] Trial 25 finished with value: 0.7450160562701532 and parameters: {'n_estimators': 2379, 'num_leaves': 1214, 'max_depth': 320, 'learning_rate': 0.0016910462182733038, 'min_child_samples': 321, 'subsample': 0.9245577554285245, 'colsample_bytree': 0.8027573271184746, 'reg_alpha': 8.590190533433615, 'reg_lambda': 6.615930936160601}. Best is trial 25 with value: 0.7450160562701532

.